## Классификация тональности текста


Обучите логистическую регрессию так, чтобы она определяла тональность текста.  

Подсчитайте величину TF-IDF для текстов. Лемматизированные тексты твитов для обучения находятся в файле tweets_lemm_train.csv. Целевой признак вы найдёте в столбце positive.

Обученной моделью классификации определите результаты предсказания для тестовой выборки твитов, которая лежит в файле tweets_lemm_test.csv. В этой выборке целевого признака нет. Сохраните предсказания в столбце positive. Таблицу с результатом сохраните как csv-файл, но чтобы тренажёр принял файл, не указывайте расширение (например, назовите файл 'predictions'). Не используйте пробелы в названии файла.

Значение accuracy вашей модели должно быть не меньше 0.62.

In [42]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = list(stopwords.words('russian'))

import numpy as np 

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score


# библиотеки для пайплайнов
# класс ColumnTransformer помогает работать с данными разного типа в одном наборе
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression

# библиотека для метода опорных векторов
from sklearn.svm import SVC

# библиотека решающих деревьев
from sklearn.tree import DecisionTreeClassifier

# библиотека для модели классификации kNN 
from sklearn.neighbors import KNeighborsClassifier

# класс для работы с пропусками
from sklearn.impute import SimpleImputer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tony_builder/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
train = pd.read_csv('https://code.s3.yandex.net/datasets/tweets_lemm_train.csv')
test= pd.read_csv('https://code.s3.yandex.net/datasets/tweets_lemm_test.csv')

for df in [train, test]:
    display(df.head())
    print()

,text,positive,lemm_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,хоть я и школотый но поверь у мы то же самый о...
1,"Да, все-таки он немного похож на него. Но мой ...",1,да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,ну ты идиотка я испугаться за ты
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,кто то в угол сидеть и погибать от голод а мы ...
4,@irina_dyshkant Вот что значит страшилка :D\r\...,1,вот что значит страшилка но блин посмотреть ве...


,text,lemm_text
0,RT @tiredfennel: если криса так интересуют дет...,если крис так интересовать ребёнок то либо они...
1,@xsealord по 200 руб. в месяц можно разместить...,по рубль в месяц можно разместить ссылка на те...
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",учитывать что сейчас преобладать один половина...
3,Товарищ :) Но я никак не могу отдельно не о...,товарищ но я никак не мочь отдельно не отметит...
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,квн быть отличный оооочень понравиться женский...


Пошагово:
- сомтрим размер выборки
- делим на целевой признак (positive [0,1]) и сам текст
- разбиваем на тренировочную и валидационную выборки

In [44]:
train.shape

(5000, 3)

In [45]:
X = train['lemm_text'].values.astype('U')
y = train.positive

In [46]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y , test_size=0.25, random_state=42)
for elem in [X_train, X_valid, y_train, y_valid]:
    print(elem.shape)

(3750,)
(1250,)
(3750,)
(1250,)


Проверив размерность, вернее ее совпадение, продолжаем:
- получаем матрицу новых признаков - значения TF-IDF как признаки.
- проверям что размерность сооветствующая: строк столько же, сколько и в выборках, а столбцы - это наши новые признаки

In [47]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
X_train_tf = count_tf_idf.fit_transform(X_train)
X_valid_tf = count_tf_idf.transform(X_valid)

for elem in X_train_tf, X_valid_tf:
    print(elem.shape)

(3750, 7975)
(1250, 7975)


In [48]:
RANDOM_STATE = 42

Готовим пайплайны с несколькими моделями:
- дерево решений
- логрег
- метод ближайших соседей
- метод опорных векторов

In [49]:
pipe_for_pred = Pipeline(
    [
        ('models', LogisticRegression(random_state=RANDOM_STATE))
    ]
) 

In [50]:
param_grid = [
    # словарь для модели DecisionTreeClassifier()
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(3, 10),
        'models__min_samples_leaf': range(3, 10),
        'models__min_samples_split': range(2, 10)
    },
    
    # словарь для модели KNeighborsClassifier() 
    {
       'models': [KNeighborsClassifier()],
        # указываем гиперпараметр модели n_neighbors
       'models__n_neighbors': range(2,5)
    },

    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE,
            solver='saga' 
            )],
        'models__C': range(1,5),
        'models__penalty': ['l1', 'l2']
    },

    # словарь для модели SVC()
    {
        'models': [SVC(kernel = 'poly') 
                    ],
        'models__degree': range(2,6)
    }
]





Запускаем рандомный поиск наилучших параметров

In [51]:
randomized_search = RandomizedSearchCV(
    pipe_for_pred, 
    param_grid, 
    cv=5,
    scoring='accuracy',
    random_state=RANDOM_STATE,
    n_jobs=-1
)

In [52]:
randomized_search.fit(X_train_tf, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('models',
                                              LogisticRegression(random_state=42))]),
                   n_jobs=-1,
                   param_distributions=[{'models': [DecisionTreeClassifier(random_state=42)],
                                         'models__max_depth': range(3, 10),
                                         'models__min_samples_leaf': range(3, 10),
                                         'models__min_samples_split': range(2, 10)},
                                        {'models': [KNeighborsClassifier()],
                                         'models__n_neighbors': range(2, 5)},
                                        {'models': [LogisticRegression(random_state=42,
                                                                       solver='saga')],
                                         'models__C': range(1, 5),
                                         'models__penalty': ['l1', 'l2']},
                                        {'models': [SVC(kernel='poly')],
                                         'models__degree': range(2, 6)}],
                   random_state=42, scoring='accuracy')

In [53]:
randomized_search.best_estimator_

Pipeline(steps=[('models',
                 DecisionTreeClassifier(max_depth=8, min_samples_leaf=9,
                                        min_samples_split=8,
                                        random_state=42))])

In [54]:
randomized_search.best_score_

0.5445333333333333

In [55]:
predictions_for_valid = randomized_search.best_estimator_.predict(X_valid_tf)
accuracy_score(y_valid, predictions_for_valid)

0.536

Получили низкое качество, работаем дальше

Запускаем жадный поиск

In [73]:
grid_search = GridSearchCV(
    pipe_for_pred, 
    param_grid, 
    cv=5,
    scoring='accuracy',
    
    n_jobs=-1
)


In [74]:
grid_search.fit(X_train_tf, y_train)


/Users/tony_builder/Library/Python/3.10/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/tony_builder/Library/Python/3.10/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/tony_builder/Library/Python/3.10/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/tony_builder/Library/Python/3.10/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/tony_builder/Library/Python/3.10/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('models',
                                        LogisticRegression(random_state=42))]),
             n_jobs=-1,
             param_grid=[{'models': [DecisionTreeClassifier(random_state=42)],
                          'models__max_depth': range(3, 10),
                          'models__min_samples_leaf': range(3, 10),
                          'models__min_samples_split': range(2, 10)},
                         {'models': [KNeighborsClassifier()],
                          'models__n_neighbors': range(2, 5)},
                         {'models': [LogisticRegression(random_state=42,
                                                        solver='saga')],
                          'models__C': range(1, 5),
                          'models__penalty': ['l1', 'l2']},
                         {'models': [SVC(kernel='poly')],
                          'models__degree': range(2, 6)}],
             scoring='accuracy')

In [75]:
grid_search.best_estimator_


Pipeline(steps=[('models',
                 LogisticRegression(C=2, random_state=42, solver='saga'))])

In [76]:

grid_search.best_score_

0.6402666666666667

In [77]:
predictions_for_valid = grid_search.best_estimator_.predict(X_valid_tf)
accuracy_score(y_valid, predictions_for_valid)

0.6168

Качество приемлемое  - на модели логрега (по заданию не менее 0.62)

Используем модель для предсказания тестовых данных

In [78]:
X_test = test['lemm_text'].values.astype('U')
X_test_tf = count_tf_idf.transform(X_test)

X_test_tf.shape


(3000, 7975)

In [81]:

predictions_for_test = grid_search.best_estimator_.predict(X_test_tf)
predictions_for_test.shape


(3000,)

In [82]:

predictions_for_test
pd.DataFrame(predictions_for_test)[0].value_counts()

1    1545
0    1455
Name: 0, dtype: int64

Записываем предсказания в отдельный файл

In [83]:
pd.DataFrame(predictions_for_test)\
    .rename(columns= {0: 'positive'})\
    .to_csv('predictions_for_test')